In [ ]:
#default_exp http

# fastcgi.http
> Not actually fastcgi at all! A very simple HTTP handler, designed for python apps behind a reverse proxy

In [ ]:
#export
from http import client,HTTPStatus
from socketserver import ThreadingTCPServer,StreamRequestHandler
from fastcore.foundation import add_docs

In [ ]:
from nbdev import *
from fastcore.utils import *
import time,socket

## Python's standard library servers

Python's `socketserver` classes call `handle` in a `BaseRequestHandler` subclass that you pass in to its constructor. You use a `BaseRequestHandler` with any of the server classes/mixins provided by `socketserver`.

In [ ]:
#export
class ReuseThreadingServer(ThreadingTCPServer): allow_reuse_address = 1

In [ ]:
show_doc(ReuseThreadingServer, title_level=3)

<h3 id="ReuseThreadingServer" class="doc_header"><code>class</code> <code>ReuseThreadingServer</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>ReuseThreadingServer</code>(**`server_address`**, **`RequestHandlerClass`**, **`bind_and_activate`**=*`True`*) :: `ThreadingTCPServer`

Mix-in class to handle each request in a new thread.

It's easiest to use `allow_reuse_address` to avoid having to wait for sockets to close, especially when testing. This class adds that functionality to `ThreadingTCPServer`.

Here's an example of using Python's standard library features along with `ReuseThreadingServer`:

In [ ]:
host,port = 'localhost',8000

In [ ]:
class _TestHandler(StreamRequestHandler):
    def handle(self):
        print('received', self.rfile.readline())
        self.wfile.write(bytes(f'pong {self.client_address[0]}\r\n', 'utf8'))

In [ ]:
@startthread
def _f():
    with ReuseThreadingServer((host,port), _TestHandler) as srv: srv.handle_request()

time.sleep(0.5) # wait for server to start

In [ ]:
c = start_client(port,host)
c.send(b'ping\r\n')
c.recv(1024)

received b'ping\r\n'


b'pong 127.0.0.1\r\n'

In [ ]:
#export
class HandlerException(Exception):
    "Class for exceptions from setup of `MinimalHTTPHandler`"
    def __init__(self, code, err=''):
        self.code = code
        super().__init__(str(err))

In [ ]:
show_doc(HandlerException, title_level=3)

<h3 id="HandlerException" class="doc_header"><code>class</code> <code>HandlerException</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>HandlerException</code>(**`code`**, **`err`**=*`''`*) :: `Exception`

Class for exceptions from setup of [`MinimalHTTPHandler`](/fastcgi/http.html#MinimalHTTPHandler)

## MinimalHTTPHandler -

In [ ]:
#export
class MinimalHTTPHandler(StreamRequestHandler):
    protocol_version,MessageClass = "HTTP/1.0",client.HTTPMessage
    def _setup(self):
        super().setup()
        self.raw_requestline = self.rfile.readline(65537)
        if len(self.raw_requestline) > 65536: raise HandlerException(HTTPStatus.REQUEST_URI_TOO_LONG)
        if not self.raw_requestline: raise HandlerException(HTTPStatus.BAD_REQUEST, "No request line")
        words = str(self.raw_requestline, 'iso-8859-1').rstrip('\r\n').split()
        if len(words) != 3: raise Exception(f'Invalid request: {words}')
        self.command,self.path,version = words
        if not version.startswith('HTTP/'): raise HandlerException( HTTPStatus.HTTP_VERSION_NOT_SUPPORTED, version )
        self.request_version = version.split('/', 1)[1]
        self.headers = client.parse_headers(self.rfile, _class=self.MessageClass)

    def setup(self):
        try: self._setup()
        except Exception as e: self.setup_ex(e)

    def setup_ex(self, e): raise e from None
    def send_header(self, keyword, value): self._headers_buffer.append(f"{keyword}: {value}\r\n")

    def end_headers(self):
        self._headers_buffer.append("\r\n")
        if self._headers_buffer: self.wfile.write("".join(self._headers_buffer).encode( 'latin-1', 'strict'))

    def send_response(self, code, message=''):
        if not message:
            try: message = HTTPStatus(code).phrase
            except ValueError: message = ''
        self._headers_buffer = [f"{self.protocol_version} {code} {message}\r\n"]
        self.send_header("Connection", "close")

In [ ]:
#export
add_docs(MinimalHTTPHandler, "A greatly simplified version of `BaseHTTPHandler`. Overriding `handle` is required.",
         setup="Overriden from `BaseRequestHandler`",
         setup_ex="Override to handle exceptions in `setup`",
         send_response="Set the HTTP response code to `code`",
         send_header="Send a MIME header to the headers buffer",
         end_headers="Send the blank line ending the MIME headers",
         MessageClass="Class used for `http.client.parse_headers")

`MinimalHTTPHandler` parses the HTTP command and headers, and sets `command`, `path`, `request_version`, and `headers`. It is based on the code in Python's `BaseHTTPHandler`, but is greatly simplified, and made consistent with the other `socketserver` servers.

To send a response, call `send_response(code)`, optionally `send_header` a few times, then `end_headers`, and finally write to `wfile`. For instance:

In [ ]:
class _TestHandler(MinimalHTTPHandler):
    def handle(self):
        print(f'Command/path/version: {self.command} {self.path} {self.request_version}')
        print(self.headers)
        self.send_response(200)
        self.send_header("Content-Type", "text/plain")
        self.send_header('Content-Length', '2')
        self.end_headers()
        self.wfile.write(b'ok')

In [ ]:
@startthread
def _f():
    with ReuseThreadingServer(('localhost',8000), _TestHandler) as httpd: httpd.handle_request()

time.sleep(0.5) # wait for server to start
test_eq(urlread("http://localhost:8000"), b'ok')

Command/path/version: GET / 1.1
Accept-Encoding: identity
Host: localhost:8000
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36
Connection: close




## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_decorator.ipynb.
Converted 02_http.ipynb.
Converted index.ipynb.
